# Домашнее задание № 1

Вам необходимо воспользоваться наработками из предыдущих частей задания (подготовка данных, сложные признаки и стратегия валидации) для предсказания оценки (```G3```) на тестовом наборе данных.

# Часть 5. Обучение алгоритма

Целевая переменная принимает дискретные значения [0; 20], поэтому вы можете поставить как задачу классификации, так и задачу регрессии.

!Не забудьте про оптимизацию параметров алгоритмов, у вас же уже есть данные для валидации.

## KNN

## Bayes

## Регрессия

## SVM

## Постройте сложную модель на основе предыдущих четырех простых и оцените ее точность

Вариант 1. Нужно взвесить предсказания простых моделей. Предложите метод взвешивания.

Вариант 2. Нужно оценить корреляцию предсказаний простых моделей.

## Сделайте итоговое предсказание

Решением домашнего задания является данный ноутбук + .csv файл с предсказанием. Пример формата файла, который должен у вас получится приведен в ```data/sample_submission.csv```. Данные для предсказания - ```X_test.csv```.

За лучшее предсказание 5 человек получат +10% к домашке.

Значения предсказаний должны быть целочисленными. Проверка будет точности решения будет по MSE.

In [ ]:
# Для jupyter notebook

submission = pd.read_csv('data/sample_submission.csv', index_col='id')
submission['G3'] = 1 # ваше предсказание
submission.to_csv('data/submission.csv', index=False) 

In [ ]:
# Для colab notebook

from google.colab import files

pd.read_csv('data/sample_submission.csv', index_col='id')
submission['G3'] = 1 # ваше предсказание
submission.to_csv('data/submission.csv', index=False)
files.download('data/submission.csv')

# Часть 6. Состязательная валидация (Adversarial Validation)

Данная стратегия часто применяется в соревнованиях по машинному обучению. Так как проверить свой алгоритм возможно только на заданном тренировочном наборе данных, а результат требуется к набору, для которого нет ответов, то выбирают следующий подход:
- оценить степень схожести тренировочных и тестовых данных;
- составить валидацию из тех тренировочных данных, которые больше похожи на тестовые.

Алгоритм реализации AV:
1. Удаляем ответы из тренировочных данных.
2. Добавляем специальные метки (```is_test```) для классов: тренировочные данные и тестовые.
То есть у тестовых данных будет значение ```is_test = 1```, а у тренировочных ```is_test = 0```.
4. Учим алгоритм классификации предсказывать заданную метку ```is_test```.
5. Делим тренировочные данные на N частей и проводим кросс-валидацию.

Обучаем N алгоритмов классификации: каждый обучается на всех тестовых данных и на N-1 наборах тренировочных. Предсказываем на 1 тренировочном наборе.

6. Для каждого элемента в выборке предсказываем вероятность принадлежности класса и сортируем тренировочные данные по вероятности принадлежности тестовой выборке.
7. Выделяем заданное число данных в валидацию.
8. Обучаемся для итогового предсказания на всех данных.

In [1]:
import pandas as pd
import numpy as np

X_train = pd.read_csv('train_features_with_answers.csv')
X_train

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G3
0,MS,M,NaN,U,LE3,T,1,3,at_home,services,...,yes,yes,4,3,3,2,3,3,0,9
1,GP,M,NaN,R,LE3,T,3,3,teacher,other,...,yes,no,3,3,4,3,5,3,16,12
2,GP,M,15.0,NaN,GT3,T,3,4,other,other,...,yes,no,5,5,1,1,1,5,0,13
3,MS,M,17.0,R,GT3,T,4,3,services,other,...,yes,yes,4,5,5,1,3,2,4,11
4,GP,F,20.0,R,GT3,T,2,1,other,other,...,yes,yes,1,2,3,1,2,2,8,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,GP,M,15.0,U,GT3,T,4,2,other,other,...,yes,no,3,3,3,1,1,3,0,10
450,GP,F,15.0,U,GT3,T,2,2,other,other,...,yes,no,5,1,2,1,1,3,4,10
451,GP,M,16.0,U,GT3,T,4,4,services,services,...,yes,no,5,3,2,1,2,5,4,15
452,MS,F,15.0,R,LE3,T,1,1,at_home,services,...,yes,no,5,4,3,1,2,4,0,10


In [2]:
COLUMNS_NOTNUM = ['school', 'sex', 'address', 'famsize', 'Pstatus',
                  'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup',
                  'famsup', 'paid', 'activities', 'nursery', 'higher',
                  'internet', 'romantic']

In [3]:
for column in COLUMNS_NOTNUM:

    X_train[column] = pd.factorize(X_train[column], sort=True)[0]

In [4]:
X_train.loc[X_train['address'] == -1, 'address'] = np.nan

In [5]:
X_train

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G3
0,1,5,NaN,1.0,1,1,1,3,0,3,...,1,1,4,3,3,2,3,3,0,9
1,0,5,NaN,0.0,1,1,3,3,4,2,...,1,0,3,3,4,3,5,3,16,12
2,0,5,15.0,NaN,0,1,3,4,2,2,...,1,0,5,5,1,1,1,5,0,13
3,1,5,17.0,0.0,0,1,4,3,3,2,...,1,1,4,5,5,1,3,2,4,11
4,0,4,20.0,0.0,0,1,2,1,2,2,...,1,1,1,2,3,1,2,2,8,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,0,5,15.0,1.0,0,1,4,2,2,2,...,1,0,3,3,3,1,1,3,0,10
450,0,4,15.0,1.0,0,1,2,2,2,2,...,1,0,5,1,2,1,1,3,4,10
451,0,5,16.0,1.0,0,1,4,4,3,3,...,1,0,5,3,2,1,2,5,4,15
452,1,4,15.0,0.0,1,1,1,1,0,3,...,1,0,5,4,3,1,2,4,0,10
